Bhuvnesh Solanki, 1811057

In [ ]:
!pip install rouge
!pip install newspaper3k

In [2]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from rouge import Rouge
import networkx as nx
from newspaper import Article
from gensim.summarization.summarizer import summarize

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
vec_path = '/content/gdrive/MyDrive/glove/glove.6B.100d.txt' # 300MB text file
embeddings_file = open(vec_path, 'r')

In [6]:
paragraph = "Subhas Chandra Bose was an Indian nationalist whose defiant patriotism made him a hero in India, but whose attempt during World War II to rid India of British rule with the help of Nazi Germany and Imperial Japan left a troubled legacy. The honorific Netaji , first applied in early 1942 to Bose in Germany by the Indian soldiers of the Indische Legion and by the German and Indian officials in the Special Bureau for India in Berlin, was later used throughout India. Bose had been a leader of the younger, radical, wing of the Indian National Congress in the late 1920s and 1930s, rising to become Congress President in 1938 and 1939. However, he was ousted from Congress leadership positions in 1939 following differences with Mahatma Gandhi and the Congress high command. He was subsequently placed under house arrest by the British before escaping from India in 1940. A 3,000-strong Free India Legion, comprising Indians captured by Erwin Rommel's Afrika Korps, was also formed to aid in a possible future German land invasion of India. By spring 1942, in light of Japanese victories in southeast Asia and changing German priorities, a German invasion of India became untenable, and Bose became keen to move to southeast Asia. Adolf Hitler, during his only meeting with Bose in late May 1942, suggested the same, and offered to arrange for a submarine. During this time Bose also became a father; his wife, or companion, Emilie Schenkl, whom he had met in 1934, gave birth to a baby girl in November 1942. Identifying strongly with the Axis powers, and no longer apologetically, Bose boarded a German submarine in February 1943. Off Madagascar, he was transferred to a Japanese submarine from which he disembarked in Japanese-held Sumatra in May 1943. Before long the Provisional Government of Free India, presided by Bose, was formed in the Japanese-occupied Andaman and Nicobar Islands. Bose had great drive and charisma—using popular Indian slogans, such as \"Jai Hind,\"—and the INA under Bose was a model of diversity by region, ethnicity, religion, and even gender. names of the two highest-ranking Indians in the councils of the British-Indian establishment, Lord Sinha of Raipur, Under Secretary of State for India and the first Indian to serve as governor of a province under the Raj, and Mr Bhupendranath Basu, a wealthy Calcutta solicitor and a member of the Council of India in London.\" Bose was eager to gain admission to a college at the University of Cambridge. However, it was already past the deadline for admission. With the help of some Indian students there and Mr. Reddaway, the Censor of Fitzwilliam Hall, a body run by the Non-Collegiate Students Board of the university, for making available the university\'s education at an economical cost without formal admission to a college, Bose entered the register of the university on 19 November 1919. He chose the Mental and Moral Sciences Tripos and simultaneously set about preparing for the Civil Service exams. On 22 June 1939 Bose organised the All India Forward Bloc a faction within the Indian National Congress, aimed at consolidating the political left, but its main strength was in his home state, Bengal. In Germany, he was attached to the Special Bureau for India under Adam von Trott zu Solz which was responsible for broadcasting on the German-sponsored Azad Hind Radio. He founded the Free India Center in Berlin, and created the Indian Legion out of Indian prisoners of war who had previously fought for the British in North Africa prior to their capture by Axis forces. Spoken as a part of a motivational speech for the Indian National Army at a rally of Indians in Burma on 4 July 1944, Bose\'s most famous quote was \"Give me blood, and I shall give you freedom!\" In this, he urged the people of India to join him in his fight against the British Raj. Spoken in Hindi, Bose's words are highly evocative. The troops of the INA were under the aegis of a provisional government, the Azad Hind Government, which came to produce its own currency, postage stamps, court and civil code, and was recognised by nine Axis states – Germany, Japan, Italian Social Republic, the Independent State of Croatia, Wang Jingwei regime in Nanjing, China, a provisional government of Burma, Manchukuo and Japanese-controlled Philippines. This attempt at conquering the Indian mainland had the Axis codename of Operation U-Go. During this operation, On 6 July 1944, in a speech broadcast by the Azad Hind Radio from Singapore, Bose addressed Mahatma Gandhi as the \"Father of the Nation\" and asked for his blessings and good wishes for the war he was fighting. Most affected were the young Tamil Indians from Malaya and Singapore, both men and women, who comprised the bulk of the civilians who had enlisted in the INA. The professional soldiers in the INA, most of whom were Punjabis, faced an uncertain future, with many fatalistically expecting reprisals from the British. In India the Indian National Congress\'s official line was succinctly expressed in a letter Mohandas Karamchand Gandhi wrote to Rajkumari Amrit Kaur. Said Gandhi, \"Subhas Bose has died well. \"Netaji Subhash Chandra Bose in order to free India from the shackles of British imperialism organized the Azad Hind Government from outside the country on October 21, 1943. Netaji set up the Provisional Government of Independent India and transferred its headquarter at Rangoon on January 7, 1944. At the outset of the war, he left India, travelling to the Soviet Union, Nazi Germany and Imperial Japan, seeking an alliance with each of them to attack the British government in India. With Imperial Japanese assistance, he re-organised and later led the Azad Hind Fauj or Indian National Army , formed with Indian prisoners-of-war and plantation workers from British Malaya, Singapore, and other parts of Southeast Asia, against British forces."

Subhas Chandra Bose was an Indian nationalist whose defiant patriotism made him a hero in India, but whose attempt during World War II to rid India of British rule with the help of Nazi Germany and Imperial Japan left a troubled legacy. The honorific Netaji , first applied in early 1942 to Bose in Germany by the Indian soldiers of the Indische Legion and by the German and Indian officials in the Special Bureau for India in Berlin, was later used throughout India. Bose had been a leader of the younger, radical, wing of the Indian National Congress in the late 1920s and 1930s, rising to become Congress President in 1938 and 1939. However, he was ousted from Congress leadership positions in 1939 following differences with Mahatma Gandhi and the Congress high command. He was subsequently placed under house arrest by the British before escaping from India in 1940. A 3,000-strong Free India Legion, comprising Indians captured by Erwin Rommel's Afrika Korps, was also formed to aid in a possible future German land invasion of India. By spring 1942, in light of Japanese victories in southeast Asia and changing German priorities, a German invasion of India became untenable, and Bose became keen to move to southeast Asia. Adolf Hitler, during his only meeting with Bose in late May 1942, suggested the same, and offered to arrange for a submarine. During this time Bose also became a father; his wife, or companion, Emilie Schenkl, whom he had met in 1934, gave birth to a baby girl in November 1942. Identifying strongly with the Axis powers, and no longer apologetically, Bose boarded a German submarine in February 1943. Off Madagascar, he was transferred to a Japanese submarine from which he disembarked in Japanese-held Sumatra in May 1943. Before long the Provisional Government of Free India, presided by Bose, was formed in the Japanese-occupied Andaman and Nicobar Islands. Bose had great drive and charisma—using popular Indian slogans, such as \"Jai Hind,\"—and the INA under Bose was a model of diversity by region, ethnicity, religion, and even gender. names of the two highest-ranking Indians in the councils of the British-Indian establishment, Lord Sinha of Raipur, Under Secretary of State for India and the first Indian to serve as governor of a province under the Raj, and Mr Bhupendranath Basu, a wealthy Calcutta solicitor and a member of the Council of India in London.\" Bose was eager to gain admission to a college at the University of Cambridge. However, it was already past the deadline for admission. With the help of some Indian students there and Mr. Reddaway, the Censor of Fitzwilliam Hall, a body run by the Non-Collegiate Students Board of the university, for making available the university\'s education at an economical cost without formal admission to a college, Bose entered the register of the university on 19 November 1919. He chose the Mental and Moral Sciences Tripos and simultaneously set about preparing for the Civil Service exams. On 22 June 1939 Bose organised the All India Forward Bloc a faction within the Indian National Congress, aimed at consolidating the political left, but its main strength was in his home state, Bengal. In Germany, he was attached to the Special Bureau for India under Adam von Trott zu Solz which was responsible for broadcasting on the German-sponsored Azad Hind Radio. He founded the Free India Center in Berlin, and created the Indian Legion out of Indian prisoners of war who had previously fought for the British in North Africa prior to their capture by Axis forces. Spoken as a part of a motivational speech for the Indian National Army at a rally of Indians in Burma on 4 July 1944, Bose\'s most famous quote was \"Give me blood, and I shall give you freedom!\" In this, he urged the people of India to join him in his fight against the British Raj. Spoken in Hindi, Bose's words are highly evocative. The troops of the INA were under the aegis of a provisional government, the Azad Hind Government, which came to produce its own currency, postage stamps, court and civil code, and was recognised by nine Axis states – Germany, Japan, Italian Social Republic, the Independent State of Croatia, Wang Jingwei regime in Nanjing, China, a provisional government of Burma, Manchukuo and Japanese-controlled Philippines. This attempt at conquering the Indian mainland had the Axis codename of Operation U-Go. During this operation, On 6 July 1944, in a speech broadcast by the Azad Hind Radio from Singapore, Bose addressed Mahatma Gandhi as the \"Father of the Nation\" and asked for his blessings and good wishes for the war he was fighting. Most affected were the young Tamil Indians from Malaya and Singapore, both men and women, who comprised the bulk of the civilians who had enlisted in the INA. The professional soldiers in the INA, most of whom were Punjabis, faced an uncertain future, with many fatalistically expecting reprisals from the British. In India the Indian National Congress\'s official line was succinctly expressed in a letter Mohandas Karamchand Gandhi wrote to Rajkumari Amrit Kaur. Said Gandhi, \"Subhas Bose has died well. \"Netaji Subhash Chandra Bose in order to free India from the shackles of British imperialism organized the Azad Hind Government from outside the country on October 21, 1943. Netaji set up the Provisional Government of Independent India and transferred its headquarter at Rangoon on January 7, 1944. At the outset of the war, he left India, travelling to the Soviet Union, Nazi Germany and Imperial Japan, seeking an alliance with each of them to attack the British government in India. With Imperial Japanese assistance, he re-organised and later led the Azad Hind Fauj or Indian National Army , formed with Indian prisoners-of-war and plantation workers from British Malaya, Singapore, and other parts of Southeast Asia, against British forces.

In [7]:
embeddings = dict()
for line in embeddings_file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float64')
    embeddings[word] = coefs

In [8]:
def clean(sentence):
    lem = WordNetLemmatizer()
    sentence = sentence.lower()
    sentence = re.sub(r'http\S+',' ',sentence)
    sentence = re.sub(r'[^a-zA-Z]',' ',sentence)
    sentence = sentence.split()
    sentence = [lem.lemmatize(word) for word in sentence if word not in stopwords.words('english')]
    sentence = ' '.join(sentence)
    return sentence

In [9]:
def average_vector(sentence):
    words = sentence.split()
    size = len(words)
    average_vector = np.zeros((size,100))
    unknown_words=[]

    for index, word in enumerate(words):
        try:
            average_vector[index] = embeddings[word].reshape(1,-1)
        except Exception as e:
            unknown_words.append(word)
            average_vector[index] = 0

    if size != 0:
        average_vector = sum(average_vector)/size
    return average_vector,unknown_words

In [10]:
def cosine_similarity(s1, s2):
    v1, _ = average_vector(s1)
    v2, _ = average_vector(s2)
    cos_sim = 0
    try:
        cos_sim = (np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    except Exception as e :
        pass
    return cos_sim

In [11]:
def page_rank(M, d = 0.85, iters = 100): #self-implemented PageRank function
  N = M.shape[1]
  col_sums = M.sum(axis=0)
  M_norm = M / col_sums[np.newaxis, :] # Normalizing along columns
  ranks = np.full((N,1), 1/N)
  M_hat = (d * M_norm + (1 - d) / N)

  for i in range(iters):
    ranks = M_hat @ ranks
  
  ranks = ranks.flatten()
  return ranks

In [12]:
sentences = sent_tokenize(paragraph)
cleaned_sentences=[]
for sentence in sentences:
    cleaned_sentences.append(clean(sentence))
similarity_matrix = np.zeros((len(cleaned_sentences),len(cleaned_sentences)))

for i in range(0,len(cleaned_sentences)):
    for j in range(0,len(cleaned_sentences)):
        if i!=j and type(cosine_similarity(cleaned_sentences[i],cleaned_sentences[j])) == np.float64 :
            similarity_matrix[i,j] = cosine_similarity(cleaned_sentences[i],cleaned_sentences[j])

In [13]:
similarity_matrix

array([[0.        , 0.89669605, 0.86672357, ..., 0.76367769, 0.93789143,
        0.880728  ],
       [0.89669605, 0.        , 0.81797652, ..., 0.83477938, 0.88975919,
        0.90670369],
       [0.86672357, 0.81797652, 0.        , ..., 0.75647581, 0.81650643,
        0.83164471],
       ...,
       [0.76367769, 0.83477938, 0.75647581, ..., 0.        , 0.79897803,
        0.8265946 ],
       [0.93789143, 0.88975919, 0.81650643, ..., 0.79897803, 0.        ,
        0.90942292],
       [0.880728  , 0.90670369, 0.83164471, ..., 0.8265946 , 0.90942292,
        0.        ]])

In [14]:
# nx_graph = nx.from_numpy_array(similarity_matrix)
# scores = nx.pagerank(nx_graph)

scores = page_rank(similarity_matrix)

In [15]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [16]:
# Specify number of sentences
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

The honorific Netaji , first applied in early 1942 to Bose in Germany by the Indian soldiers of the Indische Legion and by the German and Indian officials in the Special Bureau for India in Berlin, was later used throughout India.
He founded the Free India Center in Berlin, and created the Indian Legion out of Indian prisoners of war who had previously fought for the British in North Africa prior to their capture by Axis forces.
Subhas Chandra Bose was an Indian nationalist whose defiant patriotism made him a hero in India, but whose attempt during World War II to rid India of British rule with the help of Nazi Germany and Imperial Japan left a troubled legacy.
With Imperial Japanese assistance, he re-organised and later led the Azad Hind Fauj or Indian National Army , formed with Indian prisoners-of-war and plantation workers from British Malaya, Singapore, and other parts of Southeast Asia, against British forces.
Spoken as a part of a motivational speech for the Indian National Army

In [17]:
def textrank_summarise(paragraph, no_of_sentences): # self-implemented summarization function based on TextRank
    
    sentences = sent_tokenize(paragraph)
    cleaned_sentences=[]
    for sentence in sentences:
        cleaned_sentences.append(clean(sentence))
    similarity_matrix = np.zeros((len(cleaned_sentences),len(cleaned_sentences)))

    for i in range(0,len(cleaned_sentences)):
        for j in range(0,len(cleaned_sentences)):
            if i!=j and type(cosine_similarity(cleaned_sentences[i],cleaned_sentences[j])) == np.float64 :
                similarity_matrix[i,j] = cosine_similarity(cleaned_sentences[i],cleaned_sentences[j])
    
    # nx_graph = nx.from_numpy_array(similarity_matrix)
    # scores = nx.pagerank(nx_graph)

    scores = page_rank(similarity_matrix)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    summary = ""
    
    for i in range(no_of_sentences):
      summary+=ranked_sentences[i][1]
    
    summary = summary.strip()
    
    return summary

In [18]:

par = "Keeping with his practice of spending time with soldiers on Diwali, Prime Minister Narendra Modi on Saturday addressed troops at Longewala Post in which he asserted that India will give “prachand jawab” (fierce reply) if it is provoked. Mr. Modi said, without naming any country, that the entire world feels troubled by “expansionist” forces and expansionism shows a “distorted mindset” which belongs to 18th century. “India believes in policy of understanding others and making them understand but if an attempt to test it is made, then the country will give a fierce reply,” he asserted. His message comes amid the continuing standoff with China at Ladakh border. “No force in the world can prevent our soldiers from protecting our borders,” he said, adding that India has shown that it has strength and the political will to give a befitting reply to those challenging it. “The world now knows that India will not compromise with its interests even one bit,” he said. Also read | Adversaries may take advantage if we do not have strong armed forces: Gen. Bipin Rawat Referring to his custom of celebrating Diwali with soldiers since he assumed office in 2014, Mr. Modi said his resolve to serve and protect the country becomes stronger by spending more time with them. He also urged troops to innovate, practise yoga and learn an Indian language they don’t know from colleagues. On the occasion, he recalled the fierce fight the post had witnessed in the 1971 war against Pakistan and paid tributes to Brig Kuldeep Singh Chandpuri, a hero of the battle, saying he became “rashtra deep” with his feat of bravery. The war was also an example of exemplary coordination among Army, Navy and Air Force, he said. India had defeated Pakistan in the war, leading to the independence of Bangladesh. Keeping with his practice of spending time with soldiers on Diwali, Prime Minister Narendra Modi on Saturday addressed troops at Longewala Post in which he asserted that India will give “prachand jawab” if it is provoked. Mr. Modi said, without naming any country, that the entire world feels troubled by “expansionist” forces and expansionism shows a “distorted mindset” which belongs to 18th century. “India believes in policy of understanding others and making them understand but if an attempt to test it is made, then the country will give a fierce reply,” he asserted. “No force in the world can prevent our soldiers from protecting our borders,” he said, adding that India has shown that it has strength and the political will to give a befitting reply to those challenging it. “The world now knows that India will not compromise with its interests even one bit,” he said."
summ = textrank_summarise(par, 10)

In [19]:
def rouge(original, generated):
    rouge = Rouge()
    score = rouge.get_scores(original,generated)
    rouge1f = score[0]['rouge-1']['f']
    rouge1p = score[0]['rouge-1']['p']
    rouge1r = score[0]['rouge-1']['r']
    rouge2f = score[0]['rouge-2']['f']
    rouge2p = score[0]['rouge-2']['p']
    rouge2r = score[0]['rouge-2']['r']
    rougelf = score[0]['rouge-l']['f']
    rougelp = score[0]['rouge-l']['p']
    rougelr = score[0]['rouge-l']['r']
    
    return (rouge1f, rouge1p, rouge1r), (rouge2f, rouge2p, rouge2r), (rougelf, rougelp, rougelr)

In [20]:
(f11, precision1, recall1), (f12, precision2, recall2), (f1l, precisionl, recalll)  = rouge(par, summ)

print(f"ROUGE-1 Scores\nPrecision: {precision1}\nRecall: {recall1}\nF-1 Score: {f11}\n\n")
print(f"ROUGE-2 Scores\nPrecision: {precision2}\nRecall: {recall2}\nF-1 Score: {f12}\n\n")
print(f"ROUGE-L Scores\nPrecision: {precisionl}\nRecall: {recalll}\nF-1 Score: {f1l}")

ROUGE-1 Scores
Precision: 0.6852791878172588
Recall: 1.0
F-1 Score: 0.813253007222565


ROUGE-2 Scores
Precision: 0.6366782006920415
Recall: 0.9533678756476683
F-1 Score: 0.763485472376767


ROUGE-L Scores
Precision: 0.6852791878172588
Recall: 1.0
F-1 Score: 0.813253007222565


In [21]:
url = "https://www.theguardian.com/politics/2021/jun/04/uk-strikes-trade-deal-norway-iceland-liechtenstein-liz-truss" # Example news article
article = Article(url)
article.download()
article.parse()

In [22]:
article.title

'UK strikes trade deal with Norway, Iceland and Liechtenstein'

In [23]:
article.authors

['Daniel Boffey']

In [24]:
article.publish_date

datetime.datetime(2021, 6, 4, 0, 0)

In [25]:
nos = len(sent_tokenize(article.text))
nos

26

In [26]:
summ2 = textrank_summarise(article.text, len(sent_tokenize(article.text))//3)

In [27]:
print(summ2)

She offered words of caution, however, conceding on Friday that there would be extra costs for businesses on both sides which had been used to trading with the UK when it was an EU member state.After months of difficult talks, the comprehensive trade deal was hailed by both the UK and Norwegian governments as being pioneering in its scope and measures, with tariff-free trade in industrial goods secured.“If more British beef and cheese had come to Norway, even more Norwegian farms may have to be closed down.”

Since the UK left the single market in January, the country’s trade with Norway, Iceland and Liechtenstein has been covered by a continuity deal that covered goods but both sides had sought to further open up flows in key areas as well.David Henig, a former UK government trade official who is now director of the UK Trade Policy Project, said: “This UK-EEA free trade agreement provides better trading conditions than World Trade Organization terms, though with considerably more trad

In [28]:
type(summ2)

str

In [29]:
data = article.text.splitlines()
del data[0:4]
ftext = ' '.join(data)

In [30]:
summ3 = summarize(ftext, ratio=0.333) # Gensim's summarization function

In [31]:
print(summ3)

Under the deal, Norway has reduced its duties in 26 areas of agriculture, which includes provisions to allow a quantity of West Country farmhouse cheddar, Orkney Scottish Island cheddar, traditional Welsh caerphilly, and Yorkshire wensleydale cheese to avoid full 277% export tariffs.
Norway’s government, meanwhile, celebrated securing the same trading relationship in goods with the UK as the EU under its trade and cooperation agreement and that there will be zero duty on frozen peeled shrimp from 1 January 2023, a key export.
“The agreement entails a continuation of all previous tariff preferences for seafood and improved market access for whitefish, shrimp and several other products,” said Odd Emil Ingebrigtsen, Norway’s fisheries minister.
A key dynamic in the trade negotiations had been the UK’s demands for greater access to the Norwegian market for its meat and diary products while Norway had hoped of better terms of trade for its seafood.
As details of the deal emerged, Norwegian 

##Results

In [32]:
(f11, precision1, recall1), (f12, precision2, recall2), (f1l, precisionl, recalll)  = rouge(article.text, summ2)

print(f"ROUGE-1 Scores\nPrecision: {precision1}\nRecall: {recall1}\nF-1 Score: {f11}\n\n")
print(f"ROUGE-2 Scores\nPrecision: {precision2}\nRecall: {recall2}\nF-1 Score: {f12}\n\n")
print(f"ROUGE-L Scores\nPrecision: {precisionl}\nRecall: {recalll}\nF-1 Score: {f1l}")

ROUGE-1 Scores
Precision: 0.4669811320754717
Recall: 1.0
F-1 Score: 0.6366559442131492


ROUGE-2 Scores
Precision: 0.39248704663212436
Recall: 0.9774193548387097
F-1 Score: 0.5600739330650094


ROUGE-L Scores
Precision: 0.4669811320754717
Recall: 1.0
F-1 Score: 0.6366559442131492


In [33]:
(f11, precision1, recall1), (f12, precision2, recall2), (f1l, precisionl, recalll)  = rouge(ftext, summ3)

print(f"ROUGE-1 Scores\nPrecision: {precision1}\nRecall: {recall1}\nF-1 Score: {f11}\n\n")
print(f"ROUGE-2 Scores\nPrecision: {precision2}\nRecall: {recall2}\nF-1 Score: {f12}\n\n")
print(f"ROUGE-L Scores\nPrecision: {precisionl}\nRecall: {recalll}\nF-1 Score: {f1l}")

ROUGE-1 Scores
Precision: 0.44862155388471175
Recall: 1.0
F-1 Score: 0.6193771583541265


ROUGE-2 Scores
Precision: 0.379746835443038
Recall: 0.9818181818181818
F-1 Score: 0.5476673387768517


ROUGE-L Scores
Precision: 0.44862155388471175
Recall: 1.0
F-1 Score: 0.6193771583541265


summ2 was the summary generated using our own textrank_summarize function.
summ3 was the summary generated using the summarize function from gensim library. Both had same ratio: 0.33.
As we can see from the rouge scores above, our method has comparable summarization quality to that generated from gensim. However, the gensim function provides a more fluent and human-like summary